In [2]:
import pandas as pd
import pandas_gbq

In [3]:
# load raw data to a dataframe
df = pd.read_csv("data/yellow_cab_data.csv")

In [4]:
# data cleaning
df = df.drop_duplicates().reset_index(drop=True) #remove dups
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime']) #change to datetime dtype (was object)
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime']) #change to datetime dtype (was object)

df['trip_id'] = df.index + 1 #create trip_id
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_id
0,1,2023-10-01 00:00:00,2023-10-01 00:07:00,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35,1
1,1,2023-10-01 00:00:00,2023-10-01 00:11:00,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35,2
2,2,2023-10-01 00:00:00,2023-10-01 00:31:00,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80,3
3,2,2023-10-01 00:00:00,2023-10-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62,4
4,2,2023-10-01 00:00:00,2023-10-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2023-10-01 06:17:00,2023-10-01 06:22:00,1,0.50,-73.990898,40.750519,1,N,-73.998245,40.750462,2,5.0,0.0,0.5,0.00,0.00,0.3,5.80,99996
99996,1,2023-10-01 06:17:00,2023-10-01 06:32:00,1,3.40,-74.014488,40.718296,1,N,-73.982361,40.752529,1,14.0,0.0,0.5,2.00,0.00,0.3,16.80,99997
99997,1,2023-10-01 06:17:00,2023-10-01 06:37:00,1,9.70,-73.963379,40.774097,1,N,-73.865028,40.770512,1,29.0,0.0,0.5,8.80,5.54,0.3,44.14,99998
99998,2,2023-10-01 06:17:00,2023-10-01 06:22:00,1,0.92,-73.984901,40.763111,1,N,-73.970696,40.759148,1,5.5,0.5,0.5,1.36,0.00,0.3,8.16,99999


In [5]:
# create dim_datetime

# pickup datetime
dim_datetime = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].drop_duplicates().reset_index(drop=True)
dim_datetime['tpep_pickup_datetime'] = dim_datetime['tpep_pickup_datetime']
dim_datetime['pickup_date'] = dim_datetime['tpep_pickup_datetime'].dt.normalize()
dim_datetime['pickup_hour'] = dim_datetime['tpep_pickup_datetime'].dt.hour
dim_datetime['pickup_day'] = dim_datetime['tpep_pickup_datetime'].dt.day
dim_datetime['pickup_month'] = dim_datetime['tpep_pickup_datetime'].dt.month
dim_datetime['pickup_year'] = dim_datetime['tpep_pickup_datetime'].dt.year
dim_datetime['pickup_weekday'] = dim_datetime['tpep_pickup_datetime'].dt.weekday

# dropoff datetime
dim_datetime['tpep_dropoff_datetime'] = dim_datetime['tpep_dropoff_datetime']
dim_datetime['dropoff_date'] = dim_datetime['tpep_dropoff_datetime'].dt.normalize()
dim_datetime['dropoff_hour'] = dim_datetime['tpep_dropoff_datetime'].dt.hour
dim_datetime['dropoff_day'] = dim_datetime['tpep_dropoff_datetime'].dt.day
dim_datetime['dropoff_month'] = dim_datetime['tpep_dropoff_datetime'].dt.month
dim_datetime['dropoff_year'] = dim_datetime['tpep_dropoff_datetime'].dt.year
dim_datetime['dropoff_weekday'] = dim_datetime['tpep_dropoff_datetime'].dt.weekday

# create datetime_id
dim_datetime['datetime_id'] = dim_datetime.index + 1
dim_datetime

,tpep_pickup_datetime,tpep_dropoff_datetime,pickup_date,pickup_hour,pickup_day,pickup_month,pickup_year,pickup_weekday,dropoff_date,dropoff_hour,dropoff_day,dropoff_month,dropoff_year,dropoff_weekday,datetime_id
0,2023-10-01 00:00:00,2023-10-01 00:07:00,2023-10-01,0,1,10,2023,6,2023-10-01,0,1,10,2023,6,1
1,2023-10-01 00:00:00,2023-10-01 00:11:00,2023-10-01,0,1,10,2023,6,2023-10-01,0,1,10,2023,6,2
2,2023-10-01 00:00:00,2023-10-01 00:31:00,2023-10-01,0,1,10,2023,6,2023-10-01,0,1,10,2023,6,3
3,2023-10-01 00:00:00,2023-10-01 00:00:00,2023-10-01,0,1,10,2023,6,2023-10-01,0,1,10,2023,6,4
4,2023-10-01 00:00:00,2023-10-01 00:16:00,2023-10-01,0,1,10,2023,6,2023-10-01,0,1,10,2023,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26579,2023-10-01 06:17:00,2023-10-01 06:27:00,2023-10-01,6,1,10,2023,6,2023-10-01,6,1,10,2023,6,26580
26580,2023-10-01 06:17:00,2023-10-01 06:25:00,2023-10-01,6,1,10,2023,6,2023-10-01,6,1,10,2023,6,26581
26581,2023-10-01 06:17:00,2023-10-01 06:31:00,2023-10-01,6,1,10,2023,6,2023-10-01,6,1,10,2023,6,26582
26582,2023-10-01 06:17:00,2023-10-01 06:37:00,2023-10-01,6,1,10,2023,6,2023-10-01,6,1,10,2023,6,26583


In [6]:
# Create dim_vendor
vendor_id_type = {
    1:"Creative Mobile Technologies, LLC",
    2:"VeriFone Inc."
}
dim_vendor = df[['VendorID']].drop_duplicates().reset_index(drop=True)
dim_vendor['vendor_name'] = dim_vendor['VendorID'].map(vendor_id_type)
dim_vendor = dim_vendor[['VendorID','vendor_name']]

dim_vendor

,VendorID,vendor_name
0,1,"Creative Mobile Technologies, LLC"
1,2,VeriFone Inc.


In [7]:
# Create dim_rate_code
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}
dim_rate_code = df[['RatecodeID']].drop_duplicates().reset_index(drop=True)
dim_rate_code['rate_code_name'] = dim_rate_code['RatecodeID'].map(rate_code_type)
dim_rate_code = dim_rate_code[['RatecodeID','rate_code_name']]

dim_rate_code

,RatecodeID,rate_code_name
0,1,Standard rate
1,3,Newark
2,2,JFK
3,5,Negotiated fare
4,4,Nassau or Westchester
5,6,Group ride


In [8]:
# Create dim_pickup_location & dim_dropoff_location

# dim_pickup_location
dim_pickup_location = df[['pickup_longitude', 'pickup_latitude']].drop_duplicates().reset_index(drop=True)
dim_pickup_location['pickup_location_id'] = dim_pickup_location.index + 1
dim_pickup_location = dim_pickup_location[['pickup_location_id','pickup_latitude','pickup_longitude']] 

# dim_dropoff_location
dim_dropoff_location = df[['dropoff_longitude', 'dropoff_latitude']].drop_duplicates().reset_index(drop=True)
dim_dropoff_location['dropoff_location_id'] = dim_dropoff_location.index + 1
dim_dropoff_location = dim_dropoff_location[['dropoff_location_id','dropoff_latitude','dropoff_longitude']]

dim_dropoff_location

,dropoff_location_id,dropoff_latitude,dropoff_longitude
0,1,40.746128,-74.004265
1,2,40.733166,-74.005943
2,3,40.675770,-73.974541
3,4,40.757767,-73.969650
4,5,40.695053,-74.177170
...,...,...,...
98412,98413,40.750462,-73.998245
98413,98414,40.752529,-73.982361
98414,98415,40.770512,-73.865028
98415,98416,40.759148,-73.970696


In [9]:
# Create dim_payment_type
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
dim_payment_type = df[['payment_type']].drop_duplicates().reset_index(drop=True)
dim_payment_type['payment_type_name'] = dim_payment_type['payment_type'].map(payment_type_name)
dim_payment_type = dim_payment_type[['payment_type','payment_type_name']]

dim_payment_type

,payment_type,payment_type_name
0,1,Credit card
1,2,Cash
2,3,No charge
3,4,Dispute


In [10]:
# Create fct_trip
fct_trip = df.merge(dim_vendor, on='VendorID') \
             .merge(dim_rate_code, on='RatecodeID') \
             .merge(dim_pickup_location, on=['pickup_longitude', 'pickup_latitude']) \
             .merge(dim_dropoff_location, on=['dropoff_longitude', 'dropoff_latitude'])\
             .merge(dim_datetime, on=['tpep_pickup_datetime','tpep_dropoff_datetime']) \
             .merge(dim_payment_type, on='payment_type') \
             [['trip_id','VendorID', 'datetime_id', 'RatecodeID', 'store_and_fwd_flag', 'pickup_location_id',
               'dropoff_location_id', 'payment_type', 'trip_distance','fare_amount', 'extra', 'mta_tax', 'tip_amount',
               'tolls_amount', 'improvement_surcharge','passenger_count', 'total_amount']]

fct_trip

,trip_id,VendorID,datetime_id,RatecodeID,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,passenger_count,total_amount
0,1,1,1,1,N,1,1,1,2.50,9.0,0.5,0.5,2.05,0.0,0.3,1,12.35
1,112,1,1,1,N,112,112,1,1.20,7.0,0.5,0.5,1.00,0.0,0.3,1,9.30
2,28,2,1,1,N,28,28,1,2.17,8.5,0.5,0.5,1.96,0.0,0.3,5,11.76
3,79,2,1,1,N,79,79,1,1.04,6.5,0.5,0.5,1.56,0.0,0.3,4,9.36
4,131,2,1,1,N,131,130,1,1.74,8.0,0.5,0.5,1.86,0.0,0.3,6,11.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,87432,2,22388,1,N,85818,86120,4,0.00,-2.5,-0.5,-0.5,0.00,0.0,-0.3,1,-3.80
99996,95235,2,25295,1,N,93403,93746,4,0.31,-3.0,0.0,-0.5,0.00,0.0,-0.3,1,-3.80
99997,95765,2,25416,1,N,93919,94265,4,0.08,-4.5,0.0,-0.5,0.00,0.0,-0.3,1,-5.30
99998,88860,1,23212,3,N,87207,87512,4,19.00,70.5,0.5,0.0,0.00,10.5,0.3,1,81.80


In [11]:
# Load dimension and fact tables to BigQuery
def load_gbq_dim(df, name):
    pandas_gbq.to_gbq(df, "yellow-cab-data-engineer.dim."+name, project_id="yellow-cab-data-engineer", if_exists="replace")
def load_gbq_fct(df, name):
    pandas_gbq.to_gbq(df, "yellow-cab-data-engineer.fct."+name, project_id="yellow-cab-data-engineer", if_exists="replace")

load_gbq_dim(dim_datetime, "dim_datetime")
load_gbq_dim(dim_vendor, "dim_vendor")
load_gbq_dim(dim_rate_code, "dim_rate_code")
load_gbq_dim(dim_pickup_location, "dim_pickup_location")
load_gbq_dim(dim_dropoff_location, "dim_dropoff_location")
load_gbq_dim(dim_payment_type, "dim_payment_type")

load_gbq_fct(fct_trip,'fct_trip')

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2957.90it/s]
